## Asynchronus Programming
This notebook is based on a [youtube video](https://www.youtube.com/watch?v=t5Bo1Je9EmE)

* In synchronous programming, everything happens sequentially
* In asynchronous programming, it does not need to be sequentially
  + does not need to wait a function to finish before executing other statements
    - the function may wait for network or I/O operations
    
### note: on ubuntu 20.04, pip install nest_asyncio, and run the following commands in the first cell:
if you installed the most recent jupyter notebook, you may not need this     

`
import nest_asyncio
nest_asyncio.apply()
`

In [2]:
import nest_asyncio
nest_asyncio.apply()

### coroutine
* the following code defines a coroutine using async keyword
* we can print main() and see it is a coroutine

### async event-loop
* in python, we need to create an async event loop to run asynchronous code



In [3]:
import asyncio

async def main():
    print('abc')
    await foo("text")


async def foo(text):
    print(text)
    await asyncio.sleep(1)

loop = asyncio.get_event_loop()
loop.run_until_complete(main())

abc
text


#### In the following code:
* a task was created by asyncio.create_task(foo('text'), with the coroutine foo(text)
* when executing the task, coroutine foo give the control back when executing await asyncio.sleep(1)
* main() executed print("finished") statement
* control was given back to foo() and "text" was printed out
* asyncio.run() creates an event loop (available after python 3.7)
#### Note: asyncio.sleep() doesn't actually sleep. It hands back control and schedules a "re-call" for continuation. It works more like a yield

In [8]:
import asyncio

# create a task by asyncio.create_task(coroutine())
async def main():
    print("hello")
    task = asyncio.create_task(foo('text'))
    print("finished")
    
async def foo(text):
    print(text)
    await asyncio.sleep(1)
    
asyncio.run(main())      

hello
finished
text


#### If we want main() to wait for task to be finished and then execute print("finished") statement, we need to use await task

In [9]:
import asyncio

# create a task by asyncio.create_task(coroutine())
async def main():
    print("hello")
    task = asyncio.create_task(foo('text'))
    await task
    print("finished")
    
async def foo(text):
    print(text)
    await asyncio.sleep(1)
    
asyncio.run(main())      

hello
text
finished


#### In the following code:
* main() created a task with corutine foo()
* when executing foo(), the control was given back to main() due to `await asyncio.sleep(1)` in foo()
* when executing `await asyncio.sleep(2)` in main(), the control was given to task after `await asyncio.sleep(1)`completed
* after foo() printed "text" and completed, contorl was given to main(), and "finished" was printed

In [10]:
import asyncio

# create a task by asyncio.create_task(coroutine())
async def main():
    print("hello")
    task = asyncio.create_task(foo('text'))
    await asyncio.sleep(2)
    print("finished")
    
async def foo(text):
    print(text)
    await asyncio.sleep(1)
    
asyncio.run(main())      

hello
text
finished


#### In the following code:
* main() created a task with corutine foo()
* when executing foo(), the control was given back to main() due to `await asyncio.sleep(10)` in foo()
* when executing `await asyncio.sleep(0.5)` in main(), the control was given to task of foo()
* since foo() waits for 10 s, the control was given back to main(), and "finished" was printed
* main() completed execution and entire program is completed
* if we don't wait for task using await, the task will not be executed after sleep(10)

In [12]:
import asyncio

# create a task by asyncio.create_task(coroutine())
async def main():
    print("hello")
    task = asyncio.create_task(foo('text'))
    await asyncio.sleep(0.5)
    print("finished")
    
async def foo(text):
    print(text)
    await asyncio.sleep(10)
    print(text)
    
asyncio.run(main())      

hello
text
finished
text


#### In the following code:
* main() created a task with corutine foo()
* when executing foo(), the control was given back to main() due to `await asyncio.sleep(10)` in foo()
* when executing `await asyncio.sleep(0.5)` in main(), the control was given to task of foo()
* since foo() waits for 10 s, the control was given back to main()
* now, main() has to wait for task to complete due to `await task` statement
* afte task completed, main() completed execution and entire program is completed

In [13]:
import asyncio

# create a task by asyncio.create_task(coroutine())
async def main():
    print("hello")
    task = asyncio.create_task(foo('text'))
    await asyncio.sleep(0.5)
    await task
    print("finished")
    
async def foo(text):
    print(text)
    await asyncio.sleep(10)
    print(text)
    
asyncio.run(main())      

hello
text
text
finished


#### In the following code
* task1 and task2 were created by asyncio.creat_task(coroutine()) and started to execute
* fetch_data(), after printing 'start fetching', sleep for 2 s and gives the control to main()
* print_numbers start to execute, before fetch_data completed sleep(2)
* fetch_data() gain the control, printed out 'done fetching' and returned data to main()
* data was printed, and main() wait for task2 to complete (`await task2`)

#### Note:
* await can only be used inside a coroutine
* asyncio.create_task(coroutine()) adds the coroutine() to event loop
* to start coroutine, must create an event loop, one way is to use asyncio.run(entry_point_coroutine())

In [22]:
async def fetch_data():
    print('start fetching')
    await asyncio.sleep(2)
    print('done fetching')
    return {'data': 1}

async def print_numbers():
    for i in range(10):
        print(i)
        await asyncio.sleep(0.25)
        
async def main():
    task1 = asyncio.create_task(fetch_data())
    task2 = asyncio.create_task(print_numbers())
    
    value = await task1
    print(value)
    await task2
    
asyncio.run(main())        

start fetching
0
1
2
3
4
5
6
7
done fetching
{'data': 1}
8
9


You can also create an event loop and use run_until_complete() function

In [21]:
async def fetch_data():
    print('start fetching')
    await asyncio.sleep(2)
    print('done fetching')
    return {'data': 1}

async def print_numbers():
    for i in range(10):
        print(i)
        await asyncio.sleep(0.25)
        
async def main():
    task1 = asyncio.create_task(fetch_data())
    task2 = asyncio.create_task(print_numbers())
    
    value = await task1
    print(value)
    await task2
    
loop = asyncio.get_event_loop()
loop.run_until_complete(main())        

start fetching
0
1
2
3
4
5
6
7
done fetching
{'data': 1}
8
9


#### run asyncio using multiple threading. [reference link](https://kimmosaaskilahti.fi/blog/2021-01-03-asyncio-workers/)
* multi-threading is useful for IO-bound tasks
* multi-threading provide the workers (threads)
* tasks are executed by ascyncio using loop.run_in_executor()

In [34]:
import asyncio
import concurrent.futures
import threading
import time

MAX_WORKERS = 4

def execute_hello(ind):      
    print(f"Thread {threading.current_thread().name}: Starting task: {ind}...")
    time.sleep(1) 
    
#     if ind == 2:
#         print("BOOM!")
#         raise Exception("Boom!")

    print(f"Thread {threading.current_thread().name}: Finished task {ind}!")
    return ind    

async def main(tasks=20):
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as pool:
        loop = asyncio.get_running_loop()
        futures = [
            loop.run_in_executor(pool, execute_hello, task)
            for task in range(tasks)
        ]
        
    try:
        results = await asyncio.gather(*futures, return_exceptions=False)
    except Exception as ex:
        print("Caught error executing task", ex)
        raise
        
    print(f"Finished processing, got results: {results}")
    
asyncio.run(main())    

Thread ThreadPoolExecutor-9_0: Starting task: 0...
Thread ThreadPoolExecutor-9_1: Starting task: 1...
Thread ThreadPoolExecutor-9_2: Starting task: 2...
Thread ThreadPoolExecutor-9_3: Starting task: 3...
Thread ThreadPoolExecutor-9_0: Finished task 0!
Thread ThreadPoolExecutor-9_0: Starting task: 4...
Thread ThreadPoolExecutor-9_1: Finished task 1!Thread ThreadPoolExecutor-9_2: Finished task 2!Thread ThreadPoolExecutor-9_3: Finished task 3!

Thread ThreadPoolExecutor-9_2: Starting task: 5...

Thread ThreadPoolExecutor-9_1: Starting task: 6...
Thread ThreadPoolExecutor-9_3: Starting task: 7...
Thread ThreadPoolExecutor-9_0: Finished task 4!
Thread ThreadPoolExecutor-9_0: Starting task: 8...
Thread ThreadPoolExecutor-9_2: Finished task 5!
Thread ThreadPoolExecutor-9_2: Starting task: 9...
Thread ThreadPoolExecutor-9_1: Finished task 6!
Thread ThreadPoolExecutor-9_1: Starting task: 10...
Thread ThreadPoolExecutor-9_3: Finished task 7!
Thread ThreadPoolExecutor-9_3: Starting task: 11...
Th